In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import pandas as pd
import numpy as np
import random
import string

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00


## Section-1: Dataset Manipulation
---
The following main goals are achieved in this section:
  - Dataset is loaded
  - Uninteresting lines are deleted from the dataset
  - Lines are converted into a list of string for further processing

In [ ]:
df = pd.read_csv ('poems_dataset.csv')
df.head ()

,id,content,title,url,genre
0,139248,ngày đông se sắt lạnh trong lòng\ncó việc đi n...,SAY NẮNG,https://www.facebook.com/groups/48640773509859...,7 chu
1,139249,ôm đàn thao thức đến nữa đêm\nréo rắt cung âm ...,NaN,https://www.facebook.com/groups/17645444269765...,7 chu
2,139250,tết có người vui có kẻ buồn\nngười cười toe to...,TẾT HAI THÁI CỰC,https://www.facebook.com/groups/17645444269765...,7 chu
3,139251,đã quá ba mươi mộng lỡ làng\nđi tìm day dứt mả...,TRÁI NGANG ĐỨC HẠNH,https://www.facebook.com/groups/48640773509859...,7 chu
4,139252,mai đào nở rộ đón nàng xuân\nsợi nắng hanh vàn...,DÁNG XUÂN,https://www.facebook.com/groups/17645444269765...,7 chu


In [ ]:
genre_list = df['genre'].tolist()
genre_list = list( dict.fromkeys(genre_list) )

print(genre_list)

['7 chu', 'luc bat', '8 chu', '5 chu', '4 chu']


In [ ]:
df = df.loc[df['genre'] == 'luc bat']
print(len(df))

87605


In [ ]:
df = df.sample(frac=0.12)
print(len(df))

10513


In [ ]:
lines = df['content'].tolist ()
print(lines [:5])

['đêm qua trời nổi cơn dông\nmưa như trút nước cánh đồng làng ta\nđược tin mất bác mối già\nhọ hàng nhà mối kéo ra từng đàn\n\ncon nào cũng trắng áo tang\nkéo cả họ hàng mối để tiễn đưa\nsáng nay trời đã tạnh mưa\nhàng xóm gần gũi cũng vừa biết tin\n\nkiến vàng kiến cánh kiến kim\ntừng đàn kéo đến cũng xin chia buồn\nsườn đê mấy bác chuồn chuồn\nlượn qua lượn lại chia buồn tiễn đưa\n\nđám tang kéo tận tới trưa\nqua bờ kênh nhỏ đắp vừa hôm qua\ntiếc thương tiễn bác mối già\nhọ hàng nhà mối đưa ra ngoài đồng\n\ngò cao nghĩa địa mênh mông\nđặt nơi yên nghỉ cho ông mối già\nlùm xùm cũng mấy cây hoa\nlà nơi an táng làng ta vẫn làm', 'mỗi khi nhắc đến cái nghèo\nngười ta liền nghĩ cái vèo thiếu ăn\nít ai nghĩ chuyện xa xăm\nnghèo ngay trong cả chính tâm hồn mình\n\ncon người từ buổi sơ sinh\nnhu cầu sự sống chỉ vin sữa sùng\nsinh vật tồn tại chung chung\ncũng nhờ dinh dưỡng bổ sung hàng ngày\nnguyên lý đời sống xưa nay\nkhông hề thay đổi theo dài thời gian\nchỉ khi lão hoá suy tàn\nhay là ta

In [ ]:
# The length of lines generated based on HMM is restricted to this value.
# It is a safeguard to protect the code from a potential infinite loop
MAX_LINE_LENGTH = 8

# This is used to check the equality of two floating point values. If
# their difference is less than this value, they are considered equal
FLOAT_DELTA = 0.0001

# This special string is used to mark the end of a sentence
END_TOKEN = 'endl'

### Section-2.2: Helper Functions for Text-Parsing

In [ ]:
# Remove white-spaces and punctuations from a line and convert it
# into a list of tokens
def tokenize (line):
    baseline = line.strip ().lower ()
    tokens = ''.join ([x for x in baseline if x not in string.punctuation]).split ()
    return tokens

# Add a pairing to dictionary
def insert_link (dictionary, key, value, debug = False):
    if key not in dictionary:
        dictionary [key] = []
    if debug: print(key, dictionary [key])
    dictionary [key].append (value)

# Convert list to probability values
def to_probability (chain):
    frequencies = {}
    probabilities = {}
    num_of_words = len (chain)

    for word in chain:
        frequencies [word] = frequencies.get (word, 0) + 1

    for word, frequency in frequencies.items ():
        probabilities [word] = round (float (frequency) / num_of_words, 3)

    return probabilities

### Section-2.3: Main Function for Building the Markov Model

In [ ]:
def build_markov_model (corpus, first_order_markov_chain, second_order_markov_chain):
    # This is a dictionary of words which have been used to
    # start a line in Shakespeare's plays
    words = []

    for line in corpus:
        tokens = tokenize (line)
        num_of_tokens = len (tokens)

        for idx in range (num_of_tokens):
            token = tokens [idx]

            if idx == 0:
                words.append (token)

                # We are not interested in the first word of a
                # line since nothing precedes it
                continue

            # Populate first-order markov chain
            last_token = tokens [idx - 1]
            insert_link (first_order_markov_chain, last_token, token)

            # The second word in a line can only have a first-level
            # markov chain since there is only a single word before it
            if idx == 1:
                continue

            # The last pair of word of a line is special. We want
            # to chain it with 'END'; to help in finishing a line
            # during predicitions
            if idx == num_of_tokens - 1:
                insert_link (second_order_markov_chain, (last_token, token), END_TOKEN)

            # Populate second-order markov chain
            second_last_token = tokens [idx - 2]
            insert_link (second_order_markov_chain, (second_last_token, last_token), token)

    # Convert first-order markov chain to probability values
    for word, chain in first_order_markov_chain.items ():
        first_order_markov_chain [word] = to_probability (chain)

    # Convert first-order markov chain to probability values
    for pair, chain in second_order_markov_chain.items ():
        second_order_markov_chain [pair] = to_probability (chain)

    print ('[STATUS] Successfully built Markov Model on Corpus!\n')
    return list (set (words))

### Section-2.4: Helpers Functions for using the Markov Model for Text-Generation from the Corpus

In [ ]:
# Pick next word from the second-order markov chain. It should be the
# highest probability one. If multiple such words exist, randomly pick one
def predict_next_word (key, dictionary, debug = False):
    max_probability = 0.0
    most_probable_words = []

    for next_word, probability in dictionary.items ():
        if probability > max_probability:
            max_probability = probability
            most_probable_words = [next_word]
        elif max_probability - probability < FLOAT_DELTA:
            most_probable_words.append (next_word)

    if debug: print (key, most_probable_words)
    return random.choice (most_probable_words)

# Randomly pick a word that can follow; from the first-order markov chain
def pick_next_word (key, dictionary, debug = False):
    if debug: print(dictionary)
    return random.choice (list(dictionary.keys()))

# Generate text based on corpus
def write_line (start_word, markov_chain_one, markov_chain_two):
    line = []
    word = start_word.lower ()

    if word not in markov_chain_one.keys ():
        return 0

    line.append (word)
    next_word = pick_next_word (start_word, markov_chain_one [start_word])
    line.append (next_word)

    n = 0
    while n < MAX_LINE_LENGTH:
        next_next_word = predict_next_word ((word, next_word), markov_chain_two [(word, next_word)])

        if next_next_word == END_TOKEN:
            return ' '.join (line)

        word = next_word
        next_word = next_next_word
        line.append (next_next_word)
        n += 1

# Write a Shakespeare play of given length
def write_play (hints, mc1, mc2):
    string_generated = ""
    for word in hints:
        line = write_line (word, mc1, mc2)
        if (line):
          line += "\n"
          string_generated += line
    return string_generated

### Section-2.5: User API for Text Prediction Given a Sequence of Words

In [ ]:
def predict_next (sequence, mc1, mc2):
    # Sanity checks
    sequence = sequence.strip ()
    if (sequence == ""):
        raise ValueError('Sequence must not be an empty string. Please retry!')

    tokens = tokenize (sequence)
    line = ''
    for token in reversed (tokens):
        line = write_line (token, mc1, mc2)

        if line:
            break

    return line + "\n"

In [ ]:
%%time

# This is the first order markov chain. It chains a word
# with the word(s) that can come after it
mc_odr1 = {}

# This is the second order markov chain. It chains a pair
# of words with word(s) that can follow it
mc_odr2 = {}

words = build_markov_model (lines, mc_odr1, mc_odr2)

[STATUS] Successfully built Markov Model on Corpus!

CPU times: user 6.78 s, sys: 73.7 ms, total: 6.86 s
Wall time: 6.98 s


In [ ]:
play_length = 150
hints = [random.choice (words) for x in range (play_length)]
poem = write_play (hints, mc_odr1, mc_odr2)
print(poem)

góc bình yên
bấm mòn cả chân mây cuối trời
tạ ê chề nỗi đau trong lòng
nam đã được ghi nét vinh danh
bổng thanh tao câu từ mộc mạc chân quê
chiêm lả lướt gió ngàn khơi trập trùng
cớ chỉ rình hờn ghen với đò đêm ngày
giọt tuôn não nề tâm can
mưu con người có nhớ không
vốn người tâm huyết thiết tha mặn nồng
sợi ngất ngây tình đời
với trái địa đàng mộng mơ
trinh nương
chim ơi nắng lớn to thế này
mướp gánh dưa người quê xa mẹ hiền
một kiếp người
mồ trầm luân
chị khâu nỗi buồn
bước ngày xưa
lững liêu xiêu bóng mẹ già
dậu mồng tơi nghe mòn mỏi đợi chờ
bận người ơi có biết không
thủy bấy lâu nay ông táo về trời
gếch lên nữa mảnh trăng thề năm xưa
mùi tin người dạ thưa trăm năm
cha trọn đời
trống đồng đánh thức muôn ngàn nhớ nhung
quí của ông cha giữ gìn
mặt ao
bão cưa gẫy cành ngày nay
người sỉn say cờ bạc phát triển vững bền
ngã đau
vũ trôi xuân thì đã muộn màng
giấc mộng vàng
buôn rèm nở hoa
đốt nhà này bấy lâu nay quê hương
thầy hôm nay ngày lễ tình nhân gian
đem hồn thơ là tiếng nói tiếng

In [ ]:
import re
print(len(re.findall(r'\w+', poem)))

243


In [ ]:
import pickle
import spacy
import numpy as np
import torch
from evaluate import load
from collections import defaultdict
from nltk.util import ngrams


def compute_perplexity(all_texts_list):
    torch.cuda.empty_cache()
    perplexity = load("perplexity", module_type="metric")
    # max sequence length and batch size are limited to 256 and 2, respectively, to avoid OOM bug
    resized_all_texts_list = [text[:256] for text in all_texts_list]
    results = perplexity.compute(predictions=resized_all_texts_list, model_id="vinai/bartpho-word", device='cuda', batch_size=2)
    return results['mean_perplexity']

def compute_wordcount(all_texts_list):
    wordcount = load("word_count")
    wordcount = wordcount.compute(data=all_texts_list)
    return wordcount['unique_words']


def compute_diversity(all_texts_list):
    ngram_range = [2, 3, 4]

    tokenizer = spacy.load("en_core_web_sm").tokenizer
    token_list = []
    for sentence in all_texts_list:
        token_list.append([str(token) for token in tokenizer(sentence)])
    ngram_sets = {}
    ngram_counts = defaultdict(int)

    metrics = {}
    for n in ngram_range:
        ngram_sets[n] = set()
        for tokens in token_list:
            ngram_sets[n].update(ngrams(tokens, n))
            ngram_counts[n] += len(list(ngrams(tokens, n)))
        metrics[f'{n}gram_repitition'] = (1 - len(ngram_sets[n]) / ngram_counts[n])
    diversity = 1
    for val in metrics.values():
        diversity *= (1 - val)
    metrics['diversity'] = diversity
    return metrics

list_of_string = []
# generate words
n = 256
for i in range(150):
    play_length = 150
    hints = [random.choice (words) for x in range (play_length)]
    poem = write_play (hints, mc_odr1, mc_odr2)
    list_of_string.append(poem)
list_of_string = [i.replace("\n", ".") for i in list_of_string]
print(list_of_string[0])
list_of_string = list(filter(lambda a: a != " ", list_of_string))

perplexity = compute_perplexity(list_of_string)
wordcount = compute_wordcount(list_of_string)
diversity = compute_diversity(list_of_string)
print("Perplexity:", perplexity)
print("Wordcount:", wordcount)
print("Diversity:", diversity)


từng chuổi gió đeo nụ tình.lệnh ta ngồi bên nhau.tuần súc lăn đá ném tưng bừng đón xuân.bần lắm thay.nhốt hồn thơ là tiếng nói tiếng yêu thương.úi trời mới được thảnh thơi.rào mênh mang đất trời.vắng dễ ta quên hết những lời yêu thương.thoi mới còn thiếu những điều viển vông.ngàn ra về lòng mãi ngất ngây tình đời.ngay cha về với những ngày bên nhau.chúa thật là đáng yêu.quân người vợ yêu thương.mủi lòng.sống phù du.mít càng tương tư.xấu vẫn không quên.lë ra thì đã muộn màng.trơ cây.canh tàng đếm từng giọt đắng gõ tâm tư.bồng say sưa trong nỗi nhớ mong.nỗi mịt mù.dắt con đi khắp mọi miền quê ta.xin chắp tay con gái nhà quê hương.khoảng bình yên.trần nữ nhi khuê các nguyện thân đợi chờ.chuyện khơi đầy niềm vui.ngược long đong một đời.huế xinh tươi dịu dàng nét xuân.cải còn non còn đọng lại bên nhau.nỗi cách chia để em được bình an.màn chơi trốn tìm ngày xưa.thoáng hình hài của cha ông.muôn nhí nhảnh nết na dịu dàng nét xuân.châu không gian tĩnh lặng cô đơn.thưởng hương trà men tửu ta dìm

Some weights of the model checkpoint at vinai/bartpho-word were not used when initializing MBartForCausalLM: ['encoder.layers.5.self_attn_layer_norm.bias', 'encoder.layers.1.self_attn.q_proj.weight', 'encoder.layers.10.self_attn.out_proj.bias', 'encoder.layers.6.self_attn.q_proj.weight', 'encoder.layers.6.self_attn.v_proj.bias', 'encoder.layers.3.self_attn.v_proj.weight', 'encoder.layers.8.self_attn.q_proj.weight', 'encoder.layers.6.self_attn.k_proj.bias', 'encoder.layers.4.self_attn_layer_norm.bias', 'encoder.layers.1.final_layer_norm.weight', 'encoder.layers.5.self_attn.v_proj.bias', 'encoder.layers.10.self_attn.k_proj.bias', 'encoder.layers.10.self_attn.k_proj.weight', 'encoder.layers.11.self_attn.q_proj.bias', 'encoder.layers.4.fc2.bias', 'encoder.layers.6.fc2.weight', 'encoder.layers.1.self_attn_layer_norm.weight', 'encoder.layers.6.self_attn.out_proj.weight', 'encoder.layers.1.self_attn.out_proj.bias', 'encoder.layers.5.final_layer_norm.bias', 'encoder.layers.0.self_attn.out_proj

  0%|          | 0/75 [00:00<?, ?it/s]

Perplexity: 4099.416478678386
Wordcount: 2865
Diversity: {'2gram_repitition': 0.31836767851823766, '3gram_repitition': 0.19456755213572963, '4gram_repitition': 0.0977094651778645, 'diversity': 0.49536543406041506}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Following code demonstrates **text prediction given a sequence**: